In [ ]:
from utils import convert_hom_to_het
from torch_geometric.datasets import TUDataset

import torch
from torch_geometric.data import Data
from torch_geometric.nn import SAGEConv, Linear

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import HeteroData
from torch_geometric.nn import HeteroConv, SAGEConv, Linear

In [ ]:
# Load the ENZYMES dataset
dataset = TUDataset(root='data', name='ENZYMES')                          # This has 3 one-hot encodings
dataset_attr = TUDataset(root='data', name='ENZYMES', use_node_attr=True) # This has 3 one-hot encodings and 18 features

In [ ]:
import torch
from torch_geometric.data import Data
from torch_geometric.nn import SAGEConv, Linear

class HomogeneousGNN(nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1,-1), hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, out_channels)

    def forward(self, data: Data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        x = torch.relu(x)
        return self.lin(x)